### Creating a LLMChain from scratch (as done by langchain initially)
Creating two classes- 1. for llm 2. for prompt. That will be used by users to interact with any llm

#### Creting llm class

In [20]:
import random

In [21]:
class demoLLM:
    def __init__(self):
        print("LLM created")
    
    def predict(self, prompt):
        response_list=["Delhi is the capital of India", "AI is Artifiical Intelligence", "I am in Bangalore"]

        return {'response': random.choice(response_list)}

In [22]:
# Creating object of class
llm=demoLLM()

llm.predict("where are you?")

LLM created


{'response': 'AI is Artifiical Intelligence'}

#### Creating a class for Prompt template

In [23]:
class demoPromptTemplate:
    def __init__(self, template, input_variables):
        self.template=template
        self.input_variables=input_variables
    
    def format(self, input_dict):
        return self.template.format(**input_dict) #format function used in this line builtin for dictionary that fills the placeholders '{}' with the values

In [24]:
template=demoPromptTemplate(
    template="Write a {length} poem about {topic}",
    input_variables=['length','topic']
)

template.format({'topic':'India', 'length':'short'})

'Write a short poem about India'

Note that these classes behave how the actual llm and prompt classes will.

Now to create an application  using these two

In [25]:
prompt=template.format({'topic':'India', 'length':'short'})

In [26]:
llm=demoLLM()
llm.predict(prompt)

LLM created


{'response': 'AI is Artifiical Intelligence'}

Now creating an LLMChain to make it easier for the users to use

In [27]:
class demoLLMChain:
    def __init__(self, llm, prompt):
        self.llm=llm
        self.prompt=prompt
    
    def run(self, input_dict):
        prompt=self.prompt.format(input_dict)
        result=self.llm.predict(prompt)

        return result['response']



In [28]:
template=demoPromptTemplate(
    template="Write a {length} poem about {topic}",
    input_variables=['length','topic']
)
llm=demoLLM()

chain=demoLLMChain(llm, template)
chain.run({'topic':'India', 'length':'short'})

LLM created


'I am in Bangalore'

As can be seen, we don't have to format the prompt and .predict on llm to get the response. But creating so many such chains caused problems. And this was created as llm and prompts can't be connected by themselves as not starndardized- llm used predict() and prompt uses format()

## Making Standardized components (Runnables)
Task is to ensure all the runnables have a common method (invoke). The best way to do it is by inheriting a common class that has this method

In [ ]:
from abc import ABC, abstractmethod

#ABC is an abstract class which means it's object can't be created directly. It can only be inherited 

In [36]:
class Runnable(ABC):
    
    @abstractmethod #abstract methods act as a placeholder that should mandatorily be implemented in the clas that inherits it
    def invoke(input_data):
        pass

In [38]:
class demoLLM(Runnable):
    def __init__(self):
        print("LLM created")
    
    def invoke(self, prompt):
        response_list=["Delhi is the capital of India", "AI is Artifiical Intelligence", "I am in Bangalore"]
        return {'response': random.choice(response_list)}

    #Should we remove predict? NO! People who are already using it will get their code crashed. Instead print a warning message saying this method will be depricated, start using...
    def predict(self, prompt):
        response_list=["Delhi is the capital of India", "AI is Artifiical Intelligence", "I am in Bangalore"]

        return {'response': random.choice(response_list)}

In [37]:
class demoPromptTemplate(Runnable):
    def __init__(self, template, input_variables):
        self.template=template
        self.input_variables=input_variables

    def invoke(self, input_dict):
        return self.template.format(**input_dict)    
    
    def format(self, input_dict):
        return self.template.format(**input_dict) #format function used in this line builtin for dictionary that fills the placeholders '{}' with the values

Now both the objects- llm and prompt have same method to communicate i.e. invoke. This is helpful as these can be connected to create long chains

In [59]:
# creating a class that connects two runnables

class RunnableConnector(Runnable):
    def __init__(self, *args): # can take any number of components like prompts, models etc.
        self.components=args

    def invoke(self, input_data):
        for component in self.components: #take components one by one and give them previous' component input 
            input_data=component.invoke(input_data)
            
        return input_data

In [48]:
llm=demoLLM()
template=demoPromptTemplate(
    template="Write a {length} poem about {topic}",
    input_variables=['length','topic']
)

chain=RunnableConnector(template,llm)
chain.invoke({'topic':'India', 'length':'short'})

LLM created


{'response': 'Delhi is the capital of India'}

As can be seen, now having a single runnableconnector class is enough compared to having a NEW CHAIN FOR CONNECTING DIFFERENT TYPES OF COMPONENTS

Creating another object (bigger chains)

In [53]:
class demoStrOutputParser(Runnable):
    def __init__(self):
        pass

    def invoke(self, input_text):
        return input_text['response']

In [57]:
llm=demoLLM()
template=demoPromptTemplate(
    template="Write a {length} poem about {topic}",
    input_variables=['length','topic']
)
parser=demoStrOutputParser()
chain=RunnableConnector(template,llm, parser)
chain.invoke({'topic':'India', 'length':'short'})

LLM created


'AI is Artifiical Intelligence'

#### Combining two chains

In [62]:
prompt1=demoPromptTemplate(
    template="Write a joke about {topic}",
    input_variables=['topic']
)

prompt2=demoPromptTemplate(
    template="explain the follwing joke: {response}",
    input_variables=['response']
)
llm=demoLLM()

chain1= RunnableConnector(prompt1, llm)
chain2=RunnableConnector(prompt2,llm, parser)

merge_chain=RunnableConnector(chain1, chain2)

merge_chain.invoke({'topic':'America'})


LLM created


'AI is Artifiical Intelligence'